In [1]:
from pytorch_lightning.utilities.types import OptimizerLRScheduler

"""Class 27: Project 3: Question Answering"""

import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertForQuestionAnswering,BertTokenizer
from torch.optim import AdamW

import pandas as pd

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
train_df=pd.read_csv("qna_train.csv")
train_df.head()

,context,question,answers
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [25]:
print(len(train_df))

87599


In [24]:
test_df=pd.read_csv("qna_test.csv")
print(len(test_df))

10570


In [4]:
train_df['context'][0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [7]:
answers=train_df.iloc[0]['answers']
answers


"{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}"

In [8]:
answers=eval(answers)
print(answers)

{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


# **How to use a pretrained model for hugging face**

In [4]:
from transformers import AutoModelForQuestionAnswering,AutoTokenizer,pipeline
model_name="deepset/tinyroberta-squad2"
quesiton_answer=pipeline(
    task="question-answering",#means ki dhoroner task er jonno amra eta toiri kortesi.ei name ta ami banay deinai.ekhane task er name ta evabei dite hobe
    model=model_name,
    tokenizer=model_name#all NLP related task e tokenizer er proyojon hoy

)


Device set to use cpu


In [5]:
input={
    'question':'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
    'context':'Architecturally, the school has a Catholic character. Atop the Main Buildings gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend Venite Ad Me Omnes. Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'
}

In [6]:
output=quesiton_answer(input)
print(output)

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\.venv\Lib\site-packages\transformers\pipelines\question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.9906218047835864, 'start': 512, 'end': 538, 'answer': 'Saint Bernadette Soubirous'}


In [7]:
idx=0
context=train_df.iloc[idx]['context']
question=train_df.iloc[idx]['question']
answers=train_df.iloc[idx]['answers']
answer_start=eval(train_df.iloc[idx]['answers'])['answer_start']
print(question)
print(answers)
print(answer_start)

To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}
[515]


In [8]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokens = tokenizer.encode_plus(
                context,
                question,
                add_special_tokens=True,
                max_length=16,
                truncation=True,
                padding='max_length',
                return_tensors='pt',
                return_offsets_mapping=True
            )


In [9]:
tokens['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [10]:
tokens['input_ids']

tensor([[    0, 37848, 37471, 28108,     6,     5,   334,     2,     2,  3972,
          2661,   222,     5,  9880,  2708,     2]])

# Data

In [13]:
class QnADataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        context = self.df.iloc[idx]['context']
        question = self.df.iloc[idx]['question']
        answers = eval(self.df.iloc[idx]['answers'])['text'][0]
        answer_start = eval(self.df.iloc[idx]['answers'])['answer_start'][0]

        try:
            tokens = self.tokenizer.encode_plus(
                context,
                question,
                add_special_tokens=True,
                max_length=512,
                truncation=True,
                padding='max_length',
                return_tensors='pt',
                return_offsets_mapping=True,
            )
            """Tokenizer breaks your text into pieces → offset mapping tells you where each token came from in the original string.
            Tokens:["Hello", "world"]
            offset_mapping:[(0, 5), (6, 11)]
tokens: ['Hugging', 'Face', 'is', 'amazing']

offset_mapping:
[(0, 7), (7, 11), (12, 14), (15, 22)]

"""

        except Exception as e:
            tokens = {
                'input_ids': torch.zeros(512, dtype=torch.long),
                'attention_mask': torch.zeros(512, dtype=torch.long),
                'offset_mapping': torch.zeros((512, 2), dtype=torch.long),
            }

        input_ids = tokens['input_ids'].squeeze()
        attention_mask = tokens['attention_mask'].squeeze()
        offset_mapping = tokens['offset_mapping'].squeeze()

        start_position = 0
        end_position = 0
        answer_end = answer_start + len(answers)

        for i, (start_char, end_char) in enumerate(offset_mapping):
            if start_char <= answer_start < end_char:
                start_position = i
            if start_char < answer_end <= end_char:
                end_position = i
                break

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor([start_position], dtype=torch.long),
            'end_positions': torch.tensor([end_position], dtype=torch.long),
        }

class QnADataModule(pl.LightningDataModule):
    def __init__(self, train_data_path, test_data_path, tokenizer):
        super().__init__()
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.tokenizer = tokenizer

    def setup(self, stage=None):
        train_df = pd.read_csv(self.train_data_path)
        # train_df = train_df.sample(n=10000, random_state=42, replace=False)

        test_df = pd.read_csv(self.test_data_path)
        # test_df = test_df.sample(n=2000, random_state=42, replace=False)

        self.train_ds = QnADataset(train_df, self.tokenizer)
        self.test_ds = QnADataset(test_df, self.tokenizer)

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=4,
            shuffle=True,
        )
    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=4,
            shuffle=False,
        )

In [20]:
class QnADataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        context = self.df.iloc[idx]['context']
        question = self.df.iloc[idx]['question']
        answers = eval(self.df.iloc[idx]['answers'])['text'][0]
        answer_start = eval(self.df.iloc[idx]['answers'])['answer_start'][0]

        try:
            tokens = self.tokenizer.encode_plus(
                context,
                question,
                add_special_tokens=True,
                max_length=512,
                truncation=True,
                padding='max_length',
                return_tensors='pt',
                return_offsets_mapping=True
            )
        except Exception as e:
            tokens = {
                'input_ids': torch.zeros(512, dtype=torch.long),
                'attention_mask': torch.zeros(512, dtype=torch.long),
                'offset_mapping': torch.zeros((512, 2), dtype=torch.long),
            }

        input_ids = tokens['input_ids'].squeeze()
        attention_mask = tokens['attention_mask'].squeeze()
        offset_mapping = tokens['offset_mapping'].squeeze()

        start_position = 0
        end_position = 0
        answer_end = answer_start + len(answers)

        for i, (start_char, end_char) in enumerate(offset_mapping):
            if start_char <= answer_start < end_char:
                start_position = i
            if start_char < answer_end <= end_char:
                end_position = i
                break

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor([start_position], dtype=torch.long),
            'end_positions': torch.tensor([end_position], dtype=torch.long),
        }

class QnADataModule(pl.LightningDataModule):
    def __init__(self, train_data_path, test_data_path, tokenizer):
        super().__init__()
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.tokenizer = tokenizer

    def setup(self, stage=None):
        train_df = pd.read_csv(self.train_data_path)
        # train_df = train_df.sample(n=10000, random_state=42, replace=False)

        test_df = pd.read_csv(self.test_data_path)
        # test_df = test_df.sample(n=2000, random_state=42, replace=False)

        self.train_ds = QnADataset(train_df, self.tokenizer)
        self.test_ds = QnADataset(test_df, self.tokenizer)

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=4,
            shuffle=True,
            num_workers=4,
            pin_memory=True,
        )
    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=4,
            shuffle=False,
            num_workers=4,
            pin_memory=True,
        )

# Model

In [22]:
from transformers import AutoModelForQuestionAnswering


class QnAModel(pl.LightningModule):
    def __init__(self, model_name='deepset/roberta-base-squad2'):
        super().__init__()
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, start_positions, end_positions):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions,
        )
        return outputs

    def compute_loss(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']

        outputs = self.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions
        )
        return outputs.loss

    def training_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx)
        self.log('train_loss', loss, prog_bar=True)#“Record the value of loss under the name train_loss.”
        return loss

    def test_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx)
        self.log('test_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return AdamW(params=self.parameters(), lr=5e-5, eps=1e-8)#eps er value ta hocce learning rate er ekta thresold ..j er thek besi choto howar dorkar nai

# Training

In [16]:
from transformers import AutoTokenizer

model_name = 'deepset/roberta-base-squad2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = QnAModel()
ds_module = QnADataModule(
    train_data_path=r"qna_train.csv",
    test_data_path=r"qna_test.csv",
    tokenizer=tokenizer
)

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' 

In [23]:
trainer = pl.Trainer(
    max_epochs=1
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, ds_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
E:\PyCharmProjects\venv_manager\.venv\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name  | Type                        | Params | Mode
-------------------------------------------------------------
0 | model | RobertaForQuestionAnswering | 124 M  | eval
-------------------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
496.226   Total estimated model params size (MB)
0         Modules in train mode
227       Modules in eval mode
E:\PyCharmProjects\venv_manager\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider i

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.


# Evaluate

In [21]:
""" Zero shot learning: Using a pretrained without any fine-tuning
    Few shot learning: We use a smaller subset of our dataset to fine-tune the pretrained model.
    Fine tuning: We use custom dataset to train pretrained model.


    Result:
    Learning             |    test loss
    Zero-shot            |    3.45
    Few-shot             |    1.26 (500)
    Complete Fine-tuning |    1.15 (500)
"""
trainer.test(model=model, datamodule=ds_module)

Output()

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
